In [1]:
import numpy as np

In [2]:
num_arms = np.random.randint(2, 11)
means = np.random.uniform(0, 1, num_arms)
std_devs = np.random.uniform(0.1, 0.5, num_arms)

T = 1000

`num_arms`: generates a random number fo arms between **2** and **10**.<br>
`means`: generates a random mean between **0** and **1** for all arms.<br>
`std_devs`: generates a random standard deviation between **0.1** and **0.5** for all arms.

In [3]:
def pull_arm(arm_index, means, std_devs):
    reward = np.random.normal(means[arm_index], std_devs[arm_index])
    reward = np.clip(reward, 0, 1)
    return reward

`pull_arm(arm_index, means, std_devs)` generates a random reward for an arm based on its mean and standard deviation with an upper and lower limit of 1 and 0 respectively

In [4]:
# Explore only
max_reward = np.max(means) * T
total_reward = 0

for _ in range(T):
    rand_arm = np.random.randint(0, num_arms)
    reward = pull_arm(rand_arm, means, std_devs)
    total_reward += reward

regret = max_reward - total_reward

print("Max reward possible: " + str(max_reward))
print("Reward obtaines: " + str(total_reward))
print("Regret score: " + str(regret))

Max reward possible: 572.9568943630386
Reward obtaines: 486.71232345990273
Regret score: 86.24457090313587


**Explore only** selects a completely random arm for T iterations.

In [5]:
# Exploit only
max_reward = np.max(means) * T
total_reward = 0

initial_rewards = [0] * num_arms
for arm in range(num_arms):
    reward = pull_arm(arm, means, std_devs)
    initial_rewards[arm] = reward
    total_reward += reward

max_arm = initial_rewards.index(np.max(initial_rewards))

for _ in range(T - num_arms):
    reward = pull_arm(max_arm, means, std_devs)
    total_reward += reward

regret = max_reward - total_reward

print("Max reward possible: " + str(max_reward))
print("Reward obtaines: " + str(total_reward))
print("Regret score: " + str(regret))

Max reward possible: 572.9568943630386
Reward obtaines: 534.5179761067956
Regret score: 38.43891825624303


**Exploit only** explores each arm once, uses the highest value observed and keeps exploiting it for all instances

In [6]:
# Epsilon-greedy
epsilon = 0.1

total_reward = 0
counts = [0] * num_arms
cur_means = [0] * num_arms

for arm in range(num_arms):
    reward = pull_arm(arm, means, std_devs)
    cur_means[arm] = reward
    counts[arm] += 1
    total_reward += reward

for t in range(T - num_arms):
    if np.random.rand() < epsilon:
        chosen_arm = np.random.randint(0, num_arms)
    else:
        chosen_arm = np.argmax(cur_means)

    reward = pull_arm(chosen_arm, means, std_devs)
    total_reward += reward

    counts[chosen_arm] += 1
    cur_means[chosen_arm] += (reward - cur_means[chosen_arm]) / counts[chosen_arm]

regret = max_reward - total_reward

print("Max reward possible: " + str(max_reward))
print("Reward obtained: " + str(total_reward))
print("Regret score: " + str(regret))

Max reward possible: 572.9568943630386
Reward obtained: 543.9385006659616
Regret score: 29.018393697077045


**Epsilon-greedy** sets initial means. Then a small *epsilon* value to randomly decide when to explore. Then we continue randomly exploring or exploiting and we keep updating the mean values.

In [7]:
total_reward = 0
counts = [0] * num_arms
cur_means = [0] * num_arms

for arm in range(num_arms):
    reward = pull_arm(arm, means, std_devs)
    total_reward += reward
    counts[arm] += 1
    cur_means[arm] = reward

for t in range(num_arms, T):
    ucb_vals = [0] * num_arms
    for arm in range(num_arms):
        if counts[arm] > 0:
            ucb_vals[arm] = cur_means[arm] + np.sqrt((2 * np.log(t)) / counts[arm])
        else:
            ucb_vals[arm] = float('inf')

    chosen_arm = np.argmax(ucb_vals)

    reward = pull_arm(chosen_arm, means, std_devs)
    total_reward += reward

    counts[chosen_arm] += 1
    cur_means[chosen_arm] += (reward - cur_means[chosen_arm]) / counts[chosen_arm]

regret = max_reward - total_reward

print("Max reward possible: " + str(max_reward))
print("Reward obtained: " + str(total_reward))
print("Regret score: " + str(regret))

Max reward possible: 572.9568943630386
Reward obtained: 528.0594981343785
Regret score: 44.897396228660114


**Upper Confidence Bound (USB1)** tries to strike a balance between exploring and exploiting. The goal is to maximize a certain value which takes into account how many tmes you've explored an arm. This allows for a balance between exploring arms and exploiting them.